This notebook is about managing an event-driven backtesting/trading platform. There are plenty of posts about writing this stuff. But the code is often poorly commented, which makes it hard to get one's head around it. Here I try to figure out some of the parts that make up such a system.
In good ol' UNIX style I write small functions that do one thing and one thing only to build the proper framework.

Note that OANDA allows trading in units rather than lots, 
so that's what I will use here.

In [1]:
import matplotlib.pyplot as pl
import pandas as pd
from collections import namedtuple
import numpy as np
from decimal import Decimal, ROUND_HALF_DOWN
%matplotlib inline

In [2]:
# Using USD/CHF of 1.4555
#getcontext().prec=6
lotDict = {1: Decimal(str(1e5))}
pip = Decimal(str(1e-4))
usd_chf = Decimal(str(1.4555))
# pip value ($ per pip)
usd_chf_pip_value = pip / usd_chf * lotDict[1]  

In [3]:
usd_chf_pip_value.quantize(Decimal("0.01"), ROUND_HALF_DOWN)

Decimal('6.87')

In [4]:
# Using EUR/USD of 1.1930 is just
eur_usd = Decimal(str(1.1930))
pip / eur_usd * lotDict[1] * eur_usd
# which can simply be rewritten as:
(pip * lotDict[1]).quantize(Decimal("0.01"), ROUND_HALF_DOWN)

Decimal('10.00')

In [6]:
# long on  usd_chf buy usd, sell chf
equity= Decimal(str(10000.00))
curPair = namedtuple('curPair', ['bid', 'ask'])
usd_chf_1 = curPair(bid=Decimal(str(1.4525)), ask=Decimal(str(1.4530)))
usd_chf_2 = curPair(bid=Decimal(str(1.4550)), ask=Decimal(str(1.4555)))

In [5]:
usd_chf1 = {'bid': Decimal(str(1.4525)), 'ask': Decimal(str(1.4530))}
usd_chf2 = {'bid': Decimal(str(1.4550)), 'ask': Decimal(str(1.4555))}

In [6]:
class position:
    def __init__(self, orderType, bid, ask, pair='AUD_USD',
                 units=100000):
        self.status = 'open'
        self.side = orderType
        self.units = Decimal(str(units))
        ask = Decimal(str(ask))
        bid = Decimal(str(bid))
        self.spreadOpen = bid - ask
        self.spreadClose = False
        self.pair = pair
        if 'JPY' in pair:
            self.pip = Decimal(str(0.01))
        else:
            self.pip = Decimal(str(0.0001))
        if orderType == "long":
            self.openPrice = ask
            self.pnl = bid - ask
        else:
            self.openPrice = bid
            self.pnl = ask - bid
            
    def closePosition(self, bid, ask):
        """
        Closing a position means using the price that was not used when 
        """
        bid, ask = Decimal(str(bid)), Decimal(str(ask))
        if self.side == "long":
            closePrice = bid
            priceMove = closePrice - self.openPrice
        else:
            closePrice = ask
            priceMove = self.openPrice - closePrice
        pipMove = self.__getPips(priceMove)
        movePricePerPip = self.__getPipValue(closePrice) 
        self.pnl = (pipMove * movePricePerPip).quantize(Decimal('0.01'),
                                                       ROUND_HALF_DOWN)
        self.spreadClose += (bid - ask)
        self.status = 'closed'
    
    def __getPips(self, movement):
        """unit: pip"""
        return movement / self.pip
    
    def __getPipValue(self, price):
        """unit: $/pip"""
        return self.pip / price * self.units
    

In [7]:
p1 = position("long", 1.4525, 1.4530)

In [8]:
p1.spreadOpen

Decimal('-0.0005')

In [9]:
p1.closePosition(1.4550, 1.4555)

In [10]:
p1.pnl

Decimal('137.46')

In [11]:
p1.spread

Decimal('-0.0010')

In [81]:
p1.spread / p1.pip

Decimal('-10')

In [82]:
p2 = position("short", 1.4550, 1.4555)
p2.closePosition(1.4525, 1.453)

In [83]:
p2.pnl

Decimal('137.65')

In [85]:
p2.spread/p1.pip

Decimal('-10')